In [ ]:
import redisai
import ml2rt

t.model = torchvision.models.resnet50(pretrained=True)
t.img_t = torch.rand(1, 3, 224, 224)
t.model.eval()
t.nn_script = torch.jit.trace(t.model, t.img_t)
ml2rt.save_torch(t.nn_script, "model.pt")
t.loaded_script = ml2rt.load_model('model.pt')
t.redis = redisai.Client()
t.redis.modelset(
    'resnet', 
    redisai.Backend.torch,
    redisai.Device.cpu,
    t.loaded_script)

## 1. Loop (to run in Gear?)

In [ ]:
import twitter
import torch
import torchvision
import numpy as np
import io, urllib.request, time
import os
import PIL
import torch.nn.functional as F
import redisai
import ast, requests 

class Object(object): pass

t = Object()

t.labels = ast.literal_eval(
    requests.get('https://gist.githubusercontent.com/yrevar/942d3a0ac09ec9e5eb3a/raw/238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt').text
)

def softmax(p):
    t = np.exp(p)
    s = t.sum()
    return t / s

def get_top_5(preds):
    preds = preds.copy()
    result = np.zeros(5).astype(np.int32)
    for i in range(result.shape[0]):
        max_ind = preds.argmax()
        result[i] = int(max_ind)
        preds[max_ind] = -1
    return result

def resize_tensor(img):
    mean = torch.Tensor([0.485, 0.456, 0.406])
    std = torch.Tensor([0.229, .224, .225])
    img = img.permute(0, 3, 1, 2)
    img /= 255.0
    img = F.interpolate(img, size=(224, 224))        
    img[0] = img[0].sub_(mean[:, None, None]).div_(std[:, None, None])
    return img



t.api = twitter.Api(
    consumer_key=os.environ['TWITTER_CONSUMER_KEY'],
    consumer_secret=os.environ['TWITTER_CONSUMER_SECRET'],
    access_token_key=os.environ['TWITTER_TOKEN_KEY'],
    access_token_secret=os.environ['TWITTER_TOKEN_SECRET'])

t.t = t.api.GetUserTimeline(screen_name="alexonsoftware", 
                        count=10, 
                        include_rts=False,
                        exclude_replies=True)

t.since_id = t.t[0].id - 1

msg = Object()
msg.running = True

t.redis = redisai.Client()

while msg.running:
    msg.t = t.api.GetUserTimeline(screen_name="alexonsoftware", 
                        count=10, 
                        include_rts=False,
                        exclude_replies=True,
                        since_id=t.since_id)
    if len(msg.t) > 0:
        for msg.tweet in msg.t:
            if 'media' in msg.tweet.AsDict():
                msg.media_list = msg.tweet.AsDict()['media']
                for msg.pic in msg.media_list:
                    msg.img = PIL.Image.open(
                        io.BytesIO(
                            urllib.request.urlopen(
                                msg.pic['media_url']).read())).convert('RGB')
                    msg.img_t = torch.Tensor(
                            np.array(msg.img)).unsqueeze(0).float()
                    msg.img_t = resize_tensor(msg.img_t)
                    t.redis.tensorset('image', msg.img_t.numpy())
                    t.redis.modelrun('resnet', 'image', 'pred')
                    
                    msg.pred = t.redis.tensorget('pred')[0]
                    
                    msg.pred = softmax(msg.pred)

                    msg.ind = get_top_5(msg.pred)
                    
                    result = ''
                    for msg.p in msg.ind:
                        result += f"{t.labels[msg.p]} : {msg.pred[msg.p]}\n"
                    print(result)
                    t.api.PostUpdate(
                        status=result,
                        in_reply_to_status_id=msg.tweet.id
                    )
            t.since_id = max(t.since_id, msg.tweet.id)
    time.sleep(1)
    msg.running = True

In [ ]:
import ast

class Recognizer():
    def __init__(self):
        self.labels = ast.literal_eval(
            requests.get('https://gist.githubusercontent.com/yrevar/942d3a0ac09ec9e5eb3a/raw/238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt').text
        )
    
    def softmax(self, p):
        t = np.exp(p)
        s = t.sum()
        return t / s
    
    def predict(self, img):
        

    def get_top_5(self, preds):
        preds = preds.copy()
        result = np.zeros(5).astype(np.int32)
        for i in range(result.shape[0]):
            max_ind = preds.argmax()
            result[i] = int(max_ind)
            preds[max_ind] = -1
        return result

    def resize_tensor(img):
        mean = torch.Tensor([0.485, 0.456, 0.406])
        std = torch.Tensor([0.229, .224, .225])
        img = img.permute(0, 3, 1, 2)
        img /= 255.0
        img = F.interpolate(img, size=(224, 224))        
        img[0] = img[0].sub_(mean[:, None, None]).div_(std[:, None, None])
        return img


